In [1]:
from bs4 import BeautifulSoup
import requests

### Task 1: Get all links to the each event

In [2]:
base_url = 'http://www.ufcstats.com./statistics/events/completed'
all_urls = [base_url + f"?page={i}" for i in range(2,23)]
all_urls.insert(0,base_url)

In [3]:
def get_links_event(urls):
    """ Return each link to the UFC events"""
    links = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        info_rows = soup.find_all('tr', class_='b-statistics__table-row')[2:]
    
        for row in info_rows:
            link = row.find('a')['href']
            links.append(link)
    return links

event_urls = get_links_event(all_urls)
event_urls[:10]

['http://www.ufcstats.com/event-details/d57e6a8971b6d2bd',
 'http://www.ufcstats.com/event-details/27541033b97c076d',
 'http://www.ufcstats.com/event-details/4a35913bd9aa4161',
 'http://www.ufcstats.com/event-details/9114c8ded5ccd71d',
 'http://www.ufcstats.com/event-details/6597b611f1c32555',
 'http://www.ufcstats.com/event-details/c36e1f4fa755ffb4',
 'http://www.ufcstats.com/event-details/8c90c1563972e44d',
 'http://www.ufcstats.com/event-details/6e2b1d631832921d',
 'http://www.ufcstats.com/event-details/81b57acd6975ac06',
 'http://www.ufcstats.com/event-details/bfe95ec546692b13']

### Task 2: Get information from each event

In [24]:
def get_links_fight(urls):
    """Return each link to the fight detail"""
    links = []
    for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content)
        table = soup.find('table', class_='b-fight-details__table')
        rows = table.find_all('tr')
        for row in rows[1:]:
            links.append(row['data-link'])
    return links

fight_urls = get_links_fight(event_urls)

In [26]:
fight_urls[1]

'http://www.ufcstats.com/fight-details/2f82e8079ba354f4'